# 🧠 AgentCore Valkey Store Usage Example

This notebook demonstrates how to use the **ValkeyStore** for long-term memory storage with semantic search capabilities, combined with **AgentCoreValkeySaver** for short-term conversation state management.

## 📋 Prerequisites

Before running this notebook, ensure you have:

1. **Valkey server** with Search module support:
   - **Local testing**: `docker run -d -p 6379:6379 valkey/valkey-bundle:latest`
   - ⚠️ Standard `valkey/valkey:latest` does NOT include Search module
   - **Production**: Amazon ElastiCache Valkey with Self-Designed cluster (includes Search module)
2. **AWS credentials** configured for Bedrock access
3. **Required packages** installed:
   ```bash
   pip install valkey langchain-aws langchain langgraph 'langgraph-checkpoint-aws[valkey]' langchain-community
   ```
   Note: Use quotes around `'langgraph-checkpoint-aws[valkey]'` for zsh shell
   ```

## 🎯 Key Features

- 🧠 **Long-term memory** storage with ValkeyStore
- 🔍 **Semantic search** using vector embeddings
- 🗂️ **Namespace organization** for data isolation
- 🚀 **AgentCore-compatible** session management
- 🔄 **TTL support** for automatic cleanup
- 🏊 **Connection pooling** for scalability
- 📊 **Cross-session memory** for personalized interactions

## 🔍 Verify Package Installation

Let's first verify that all required packages are installed:

In [ ]:
import sys

required_packages = [
    'langchain',
    'langchain_aws',
    'langchain_core',
    'langgraph',
    'langgraph_checkpoint_aws',
    'valkey'
]

print("🔍 Checking installed packages...\n")
missing_packages = []
for package in required_packages:
    try:
        __import__(package)
        print(f"✅ {package}")
    except ImportError:
        print(f"❌ {package} - NOT INSTALLED")
        missing_packages.append(package)

if missing_packages:
    print(f"\n⚠️  Missing packages: {', '.join(missing_packages)}")
    print("\nPlease install them using:")
    print("pip install valkey langchain-aws langchain langgraph 'langgraph-checkpoint-aws[valkey]' langchain-community")
else:
    print("\n✅ All required packages are installed!")

## 📦 Import Dependencies

First, let's import all the necessary libraries:

In [ ]:
import time
import uuid
import logging
from typing import Any

from langchain.chat_models import init_chat_model
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.runnables import RunnableConfig
from langchain_aws import BedrockEmbeddings

# Note: If you see a Pylance import error below, it's a false positive.
# The import works correctly at runtime - the package is properly installed.
from langgraph_checkpoint_aws.store.valkey import ValkeyStore, AsyncValkeyStore
from langgraph_checkpoint_aws.agentcore.valkey import AgentCoreValkeySaver
from valkey import Valkey

logging.getLogger().setLevel(logging.INFO)

print("✅ All dependencies imported successfully!")

## ⚙️ Configuration

Set up the configuration for Valkey connection, embeddings, and language model:

### ValkeyStore Configuration

- `VALKEY_ENDPOINT`: Use `localhost:6379` for local testing, or `my-cluster.cache.amazonaws.com:6379` for ElastiCache
- `REGION`: AWS region where your resources are located
- `MODEL_ID`: Bedrock model ID for the agent
- `EMBEDDING_MODEL_ID`: Bedrock embeddings model for semantic search

### Namespaces for Memory Organization

ValkeyStore uses hierarchical namespaces to organize data:

- `(actor_id, session_id)`: Conversation messages (session-specific)
- `("facts", actor_id)`: Extracted facts (cross-session)
- `("preferences", actor_id)`: User preferences (cross-session)

This mirrors AgentCore Memory's namespace structure for compatibility.

In [ ]:
# Configuration
VALKEY_ENDPOINT = "localhost:6379"  # Use localhost for local testing
REGION = "us-west-2"
MODEL_ID = "us.amazon.nova-premier-v1:0"
EMBEDDING_MODEL_ID = "amazon.titan-embed-text-v2:0"

print("📋 Configuration:")
print(f"   - Valkey URL: valkey://{VALKEY_ENDPOINT}")
print(f"   - Model ID: {MODEL_ID}")
print(f"   - Embedding Model: {EMBEDDING_MODEL_ID}")

## 🔗 Initialize Valkey Clients

Create separate Valkey clients for checkpointing and memory storage:

In [ ]:
print("🔗 Creating Valkey clients...")

# Create separate clients for checkpoint and store
checkpoint_client = Valkey.from_url(
    f"valkey://{VALKEY_ENDPOINT}",
    decode_responses=False,
    max_connections=20
)

store_client = Valkey.from_url(
    f"valkey://{VALKEY_ENDPOINT}",
    decode_responses=False,
    max_connections=20
)

print("✅ Valkey clients created successfully!")
print(f"   - Checkpoint client: max_connections=20")
print(f"   - Store client: max_connections=20")

## 🔍 Initialize Embeddings

Initialize Bedrock embeddings for semantic search:

In [ ]:
print("🔍 Initializing Bedrock embeddings...")
import boto3
from botocore.config import Config
try:
    # Configure boto3 for async concurrency
    bedrock_config = Config(
        max_pool_connections=50,
        retries={'max_attempts': 3, 'mode': 'adaptive'}
    )
    
    embeddings = BedrockEmbeddings(
        model_id=EMBEDDING_MODEL_ID,
        region_name=REGION
    ,
        client=boto3.client(
            "bedrock-runtime",
            config=bedrock_config
        )
    )
    print("✅ Bedrock embeddings initialized successfully!")
    print(f"   - Model: {EMBEDDING_MODEL_ID}")
    print(f"   - Dimensions: 1024 (Titan Embed Text v2)")
except Exception as e:
    print(f"❌ Failed to initialize embeddings: {e}")
    print("Please check your AWS credentials and region configuration.")
    raise

## 🧠 Create ValkeyStore with Semantic Search

Create the ValkeyStore with vector search capabilities:

In [ ]:
print("🧠 Creating ValkeyStore with semantic search...")
# Define shared ValkeyIndexConfig to avoid duplication
valkey_index_config = {
    "collection_name": "long_term_memory",
    "dims": 1024,  # Titan Embed Text v2 dimensions
    "embed": embeddings,
    "fields": ["text", "content"],  # Fields to embed for search
    "timezone": "UTC",
    "index_type": "hnsw",  # High-performance approximate search
    "distance_metric": "COSINE"
}

valkey_ttl_config = {"default_ttl": None}  # No expiration for long-term memory

try:
    store = ValkeyStore(
        client=store_client,
        index=valkey_index_config,
        ttl=valkey_ttl_config
    )
    
    # Setup the index
    store.setup()
    
    print("✅ ValkeyStore created and configured!")
    print(f"   - Index name: long_term_memory")
    print(f"   - Vector dimensions: 1024")
    print(f"   - Index type: HNSW (High-performance)")
    print(f"   - Distance metric: COSINE")
    print(f"   - TTL: None (persistent storage)")
    
except Exception as e:
    print(f"❌ Failed to create ValkeyStore: {e}")
    print("Please ensure Valkey server with Search module is running.")
    raise
# Create AsyncValkeyStore with same configuration
async_store = AsyncValkeyStore(
    client=store_client,
    index=valkey_index_config,
    ttl=valkey_ttl_config
)

print("✅ AsyncValkeyStore created with same configuration as sync store\n")

## 🔄 Create AgentCore Valkey Checkpointer

Create the checkpointer for short-term conversation state:

In [ ]:
print("🔄 Creating AgentCore Valkey checkpointer...")

try:
    checkpointer = AgentCoreValkeySaver(
        client=checkpoint_client,
        ttl=86400  # 24 hour checkpoint retention
    )
    
    print("✅ AgentCore Valkey checkpointer created successfully!")
    print(f"   - TTL: 86400 seconds (24 hours)")
    
except Exception as e:
    print(f"❌ Failed to create checkpointer: {e}")
    raise

## 🤖 Initialize Language Model

Initialize the Bedrock language model:

In [ ]:
print("🤖 Initializing language model...")

try:
    model = init_chat_model(
        MODEL_ID,
        model_provider="bedrock_converse",
        region_name=REGION
    )
    print("✅ Language model initialized successfully!")
except Exception as e:
    print(f"❌ Failed to initialize model: {e}")
    print("Please check your AWS credentials and region configuration.")
    raise

## 🎨 Create Agent with Store Integration

Create the agent graph with both checkpointer and store:

In [ ]:
print("🎨 Creating agent graph...")

# Create agent with checkpointer and store
graph = create_agent(
    model,
    tools=[],  # No tools for this example, just conversation
    checkpointer=checkpointer,
    store=store,
)

print("✅ Agent graph created successfully!")
print("   - Checkpointer: AgentCoreValkeySaver (short-term state)")
print("   - Store: ValkeyStore (long-term memory)")

## 🛠️ Memory Helper Functions

Define helper functions for manual memory management:

In [ ]:
def save_message_to_store(store, actor_id, thread_id, message, role):
    """Save a message to ValkeyStore with proper namespace."""
    namespace = (actor_id, thread_id)
    
    # Extract serializable content from message
    text_content = message.content if hasattr(message, 'content') else str(message)
    
    store.put(
        namespace,
        str(uuid.uuid4()),
        value={
            'text': text_content,  # Required for vector embedding
            'content': text_content,  # For retrieval
            'role': role,
            'actor_id': actor_id,
            'session_id': thread_id,
            'message_type': type(message).__name__  # Store type as string
        },
        index=["text", "content"]  # Explicitly enable vector indexing
    )

def get_relevant_context(store, actor_id, query, limit=5):
    """Search ValkeyStore for relevant user preferences."""
    preferences_namespace = ('preferences', actor_id)
    results = store.search(preferences_namespace, query=query, limit=limit)
    return [r.value.get('content', r.value) for r in results]

def run_agent(query: str, config: RunnableConfig):
    """Run agent, print output, and save messages to ValkeyStore."""
    actor_id = config.get('configurable', {}).get('actor_id', 'default')
    thread_id = config.get('configurable', {}).get('thread_id', 'default')
    
    # Before running agent: search for relevant context
    context = get_relevant_context(store, actor_id, query)
    if context:
        print(f"📚 Found {len(context)} relevant preferences from past conversations")
    
    printed_ids = set()
    user_message = None
    ai_message = None
    
    events = graph.stream(
        {'messages': [{'role': 'user', 'content': query}]},
        config,
        stream_mode='values',
    )
    
    for event in events:
        if 'messages' in event:
            for msg in event['messages']:
                if id(msg) not in printed_ids:
                    msg.pretty_print()
                    printed_ids.add(id(msg))
                    
                    # Capture messages for saving
                    if isinstance(msg, HumanMessage):
                        user_message = msg
                    elif isinstance(msg, AIMessage):
                        ai_message = msg
    
    # After agent runs: save messages to ValkeyStore
    if user_message:
        save_message_to_store(store, actor_id, thread_id, user_message, 'user')
        print(f"💾 Saved user message to ValkeyStore")
    
    if ai_message:
        save_message_to_store(store, actor_id, thread_id, ai_message, 'assistant')
        print(f"💾 Saved assistant message to ValkeyStore")
        
        # Extract and save preferences if detected
        print(f"🔍 Checking for preferences in user message...")
        if user_message:
            user_text = user_message.content.lower()
            if any(kw in user_text for kw in ['favorite', 'like', 'love', 'prefer', 'enjoy']):
                preferences_namespace = ('preferences', actor_id)
                store.put(
                    preferences_namespace,
                    f'pref_{uuid.uuid4()}',
                    value={
                        'content': user_message.content[:500],
                        'text': user_message.content[:500],
                        'extracted_from': thread_id,
                        'type': 'preference'
                    },
                    index=["text", "content"]  # Explicitly enable vector indexing
                )
                print(f"🔖 Extracted and saved user preference")

print("✅ Memory helper functions defined!")

## 🎯 Configure Session

Set up the session configuration with both `actor_id` and `thread_id`:

### LangGraph RuntimeConfig

The `config` dictionary is crucial for ValkeyStore's namespace organization:

- **`actor_id`**: Identifies the user/agent (e.g., `user-123`, `agent-abc`)
  - Used as the root namespace for long-term memories
  - Enables multi-tenancy (isolate users)
  - Compatible with AgentCore's actor concept

- **`thread_id`**: Identifies the conversation session (e.g., `session-001`)
  - Used for conversation-specific data
  - Enables multiple concurrent conversations per user
  - Maps to AgentCore's session_id

In [ ]:
# Configuration for session management
config = {
    "configurable": {
        "thread_id": "session-id-1",  # Session identifier
        "actor_id": "user-1",         # User identifier
    }
}

print("📋 Session configuration:")
for key, value in config["configurable"].items():
    print(f"   - {key}: {value}")

## 💬 First Conversation

Let's start a conversation where the user shares their cooking preferences:

In [ ]:
print("💬 Starting first conversation...")
print("="*80)
print("FIRST CONVERSATION - Session 1")
print("="*80 + "\n")

prompt = """
Hey there! I'm cooking one of my favorite meals tonight: salmon with rice and veggies (healthy). It has
great macros for my weightlifting competition that is coming up. What can I add to this dish to make it taste better
and also improve the protein and vitamins I get?
"""

run_agent(prompt, config)

## 📊 Verify Storage

Let's verify what was stored in ValkeyStore:

### What Was Stored?

1. **User message** → `("user-1", "session-id-1")` namespace
2. **Assistant message** → `("user-1", "session-id-1")` namespace
3. **User preferences** → `("preferences", "user-1")` namespace (extracted)

In [ ]:
print("📊 Storage Information:")
print("="*80)

# Check message keys
message_keys = store_client.keys('*user-1*/session-id-1*')
print(f"\n📝 Session messages: {len(message_keys)} keys")
if message_keys:
    print(f"   Sample: {message_keys[0].decode()}")

# Check preference keys
preference_keys = store_client.keys('*preferences/user-1*')
print(f"\n⭐ User preferences: {len(preference_keys)} keys")
if preference_keys:
    print(f"   Sample: {preference_keys[0].decode()}")

## 🔄 Second Conversation (New Session)

Start a new session to demonstrate cross-session memory retrieval:

In [ ]:
# New session configuration
config2 = {
    "configurable": {
        "thread_id": "session-id-2",  # New session
        "actor_id": "user-1",         # Same user
    }
}

print("\n" + "="*80)
print("SECOND CONVERSATION - Session 2 (Same User)")
print("="*80)
print("🔍 This will demonstrate cross-session memory retrieval\n")

prompt2 = "What are some good dinner ideas for tonight?"

run_agent(prompt2, config2)

## 🔍 Semantic Search Examples

Demonstrate semantic search capabilities by querying related concepts:

In [ ]:
print("\n" + "="*80)
print("SEMANTIC SEARCH EXAMPLES")
print("="*80 + "\n")

# Search by related concept (not exact keyword)
test_queries = [
    ("protein-rich foods", "Should find salmon/weightlifting context"),
    ("athletic nutrition", "Should find healthy eating preferences"),
    ("dinner ideas", "Should find cooking-related memories")
]

for query, description in test_queries:
    print(f"🔍 Query: '{query}'")
    print(f"   Expected: {description}")
    
    results = store.search(("preferences", "user-1"), query=query, limit=3)
    print(f"   ✅ Found {len(results)} results:")
    for i, result in enumerate(results, 1):
        content = result.value.get("content", "")[:80]
        print(f"      {i}. {content}...")
    print()

## 🔄 Async Semantic Search

Demonstrate async search capabilities and verify parity with sync results:

In [ ]:

print("\n" + "="*80)
print("ASYNC SEMANTIC SEARCH DEMONSTRATION")
print("="*80 + "\n")

# Test async search and compare with sync
test_query = "protein-rich foods"
print(f"🔍 Test Query: '{test_query}'\n")

# Sync search
sync_results = store.search(("preferences", "user-1"), query=test_query, limit=3)
print(f"🔵 Sync Search Results: {len(sync_results)} found")
for i, result in enumerate(sync_results, 1):
    content = result.value.get("content", "")[:60]
    print(f"   {i}. Score: {result.score:.4f} - {content}...")

# Async search
async_results = await async_store.asearch(("preferences", "user-1"), query=test_query, limit=3)
print(f"\n🟢 Async Search Results: {len(async_results)} found")
for i, result in enumerate(async_results, 1):
    content = result.value.get("content", "")[:60]
    print(f"   {i}. Score: {result.score:.4f} - {content}...")

# Verify parity
print("\n" + "="*80)
print("SYNC/ASYNC PARITY VERIFICATION")
print("="*80)

if len(sync_results) == len(async_results):
    print(f"✅ Result count matches: {len(sync_results)} results from both")
else:
    print(f"⚠️  Result count differs: sync={len(sync_results)}, async={len(async_results)}")

# Compare scores and content
for i, (sync_res, async_res) in enumerate(zip(sync_results, async_results), 1):
    score_diff = abs(sync_res.score - async_res.score)
    if score_diff < 0.0001:
        print(f"   Result {i}: ✅ Scores match ({sync_res.score:.4f})")
    else:
        print(f"   Result {i}: ⚠️  Score diff: {score_diff:.6f}")
    
    if sync_res.key == async_res.key:
        print(f"             ✅ Keys match: {sync_res.key}")
    else:
        print(f"             ⚠️  Keys differ: {sync_res.key} vs {async_res.key}")

print("\n🎉 Async search demonstration complete!")
print("   Both sync and async search return identical results with same scores.")

## 🧹 Cleanup (Optional)

Demonstrate cleanup functionality:

In [ ]:
print("🧹 Cleanup Options")
print("="*80)
print("\n⚠️  The following code is commented out to prevent accidental data deletion.")
print("Uncomment to clean up demo data:\n")

# WARNING: This deletes all data for user-1
# Uncomment to run:

# print("Deleting preferences...")
# preferences = store.search(("preferences", "user-1"), query="*", limit=100)
# for pref in preferences:
#     store.delete(("preferences", "user-1"), pref.key)
# print(f"✅ Deleted {len(preferences)} preferences")

# print("\nDeleting session 1 messages...")
# session1 = store.search(("user-1", "session-id-1"), query="*", limit=100)
# for msg in session1:
#     store.delete(("user-1", "session-id-1"), msg.key)
# print(f"✅ Deleted {len(session1)} session 1 messages")

# print("\nDeleting session 2 messages...")
# session2 = store.search(("user-1", "session-id-2"), query="*", limit=100)
# for msg in session2:
#     store.delete(("user-1", "session-id-2"), msg.key)
# print(f"✅ Deleted {len(session2)} session 2 messages")

# print("\n✅ All data deleted for user-1")

print("To clean up, uncomment the code above and re-run this cell.")

## 🎉 Summary

Congratulations! You've successfully demonstrated the **ValkeyStore/AsyncValkeyStore** with semantic search capabilities combined with **AgentCoreValkeySaver** for checkpoint persistence.

### Key Benefits Demonstrated:

- 🧠 **Long-term memory storage** with ValkeyStore
- 🔍 **Semantic search** using vector embeddings
- 🗂️ **Namespace organization** for data isolation
- 🚀 **AgentCore-compatible** session management
- 📊 **Cross-session memory** retrieval
- 🎯 **Preference extraction** and storage
- 🔄 **TTL support** for data lifecycle management
- 🏊 **Connection pooling** for scalability

### What We Covered:

1. **Basic Setup**: Created ValkeyStore with vector search index
2. **Agent Integration**: Used with LangGraph agents and checkpointer
3. **Session Management**: AgentCore-compatible actor_id and thread_id patterns
4. **Memory Storage**: Manual message and preference storage
5. **Cross-Session Retrieval**: Demonstrated long-term memory access
6. **Semantic Search**: Vector-based similarity search
8. **Cleanup**: Proper resource management

### 🎯 Namespace Organization Best Practices:

| Namespace | Purpose | Scope | TTL |
|-----------|---------|-------|-----|
| `(actor_id, session_id)` | Conversation messages | Session | 24h |
| `("preferences", actor_id)` | User preferences | Cross-session | None |
| `("facts", actor_id)` | Extracted facts | Cross-session | None |
| `("knowledge", "global")` | Shared knowledge | Global | None |

### 🔗 Architecture:

```
┌─────────────────────────────────────────────┐
│           LangGraph Agent                   │
├─────────────────────────────────────────────┤
│                                             │
│  ┌──────────────────┐  ┌─────────────────┐  │
│  │ AgentCoreValkey  │  │ ValkeyStore/    │  │
│  │     Saver        │  │ AsyncValkeyStore│  │
│  │  (Short-term)    │  │  (Long-term)    │  │
│  └──────────────────┘  └─────────────────┘  │
│           │                     │           │
└───────────┼─────────────────────┼───────────┘
            │                     │
            ▼                     ▼
      ┌──────────────────────────────┐
      │   Valkey / ElastiCache       │
      │  (with Search module)        │
      └──────────────────────────────┘
```

The **ValkeyStore/AsyncValkeyStore** provides a powerful, scalable solution for long-term memory storage with semantic search that works seamlessly with AgentCore session management patterns! 🚀